In [ ]:
from datetime import datetime, timezone
from typing import Dict

import pyspark.sql.functions as F
from pyspark.sql import DataFrame


CATALOG = ""

VOLUME_CATALOG = "main"
VOLUME_SCHEMA = "engenharia_dados"
VOLUME_NAME = "aviacao_landing"

LANDING_CSV_BASE_PATH = f"/Volumes/{VOLUME_CATALOG}/{VOLUME_SCHEMA}/{VOLUME_NAME}/aviacao/landing"

LANDING_SCHEMA = "aviacao_landing"
META_SCHEMA = "aviacao_meta"

JDBC_URL = "jdbc:postgresql://aviao-metrics-databricks.c5kyywe4w1hd.us-east-1.rds.amazonaws.com:5432/postgres"
JDBC_USER = "postgres"
JDBC_PASSWORD = "0mZhbgsBlOtYIUW3KNXq"
JDBC_DRIVER = "org.postgresql.Driver"

TABLE_CONFIGS: Dict[str, Dict] = {
    "companhias_aereas": {"schema": "aviacao", "business_key": ["id"]},
    "modelos_avioes": {"schema": "aviacao", "business_key": ["id"]},
    "aeroportos": {"schema": "aviacao", "business_key": ["id"]},
    "aeronaves": {"schema": "aviacao", "business_key": ["id"]},
    "funcionarios": {"schema": "aviacao", "business_key": ["id"]},
    "clientes": {"schema": "aviacao", "business_key": ["id"]},
    "voos": {"schema": "aviacao", "business_key": ["id"]},
    "reservas": {"schema": "aviacao", "business_key": ["id"]},
    "bilhetes": {"schema": "aviacao", "business_key": ["id"]},
    "bagagens": {"schema": "aviacao", "business_key": ["id"]},
    "manutencoes": {"schema": "aviacao", "business_key": ["id"]},
    "tripulacao_voo": {"schema": "aviacao", "business_key": ["id"]},
}


def qname(schema: str, table: str) -> str:
    if CATALOG:
        return f"{CATALOG}.{schema}.{table}"
    return f"{schema}.{table}"


def now_utc() -> datetime:
    return datetime.now(timezone.utc)


def is_df_empty(df: DataFrame) -> bool:
    return df.limit(1).count() == 0


def init_schema(schema_name: str) -> None:
    schema_qualified = f"{CATALOG}.{schema_name}" if CATALOG else schema_name
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_qualified}")


def init_watermark_table() -> None:
    init_schema(META_SCHEMA)

    wm_table = qname(META_SCHEMA, "watermark_incremental")
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {wm_table} (
            tabela STRING NOT NULL,
            ultima_data_ref TIMESTAMP NOT NULL,
            ultima_execucao_ts TIMESTAMP NOT NULL,
            CONSTRAINT pk_watermark_incremental PRIMARY KEY (tabela)
        )
        USING DELTA
    """)


def get_watermark(table_name: str) -> datetime:
    wm_table = qname(META_SCHEMA, "watermark_incremental")
    df = spark.table(wm_table).filter(F.col("tabela") == table_name)

    if is_df_empty(df):
        return datetime(1900, 1, 1, tzinfo=timezone.utc)

    row = df.select("ultima_data_ref").head(1)[0]
    return row["ultima_data_ref"]


def update_watermark(table_name: str, new_data_ref: datetime) -> None:
    wm_table = qname(META_SCHEMA, "watermark_incremental")
    ts_exec = now_utc()

    (
        spark.table(wm_table)
        .filter(F.col("tabela") != table_name)
        .createOrReplaceTempView("tmp_wm_others")
    )

    others = spark.table("tmp_wm_others")

    new_row = spark.createDataFrame(
        [(table_name, new_data_ref, ts_exec)],
        ["tabela", "ultima_data_ref", "ultima_execucao_ts"],
    )

    merged = others.unionByName(new_row, allowMissingColumns=True)
    merged.write.mode("overwrite").format("delta").saveAsTable(wm_table)


def build_incremental_query(pg_schema: str, table_name: str, last_wm: datetime) -> str:
    last_wm_str = last_wm.strftime("%Y-%m-%d %H:%M:%S")

    return f"""
        SELECT
            t.*,
            COALESCE(t.AUD_DH_ALTERACAO, t.AUD_DH_CRIACAO) AS data_ref,
            CASE
                WHEN t.AUD_DH_ALTERACAO IS NULL THEN 'I'
                ELSE 'U'
            END AS change_op
        FROM {pg_schema}.{table_name} t
        WHERE COALESCE(t.AUD_DH_ALTERACAO, t.AUD_DH_CRIACAO) > TIMESTAMP '{last_wm_str}'
        ORDER BY data_ref, id
    """


def extract_incremental_from_postgres(table_name: str, conf: Dict, last_wm: datetime) -> DataFrame:
    pg_schema = conf["schema"]
    query = build_incremental_query(pg_schema, table_name, last_wm)

    print(f"[{table_name}] Executando query incremental a partir de {last_wm} ...")

    df = (
        spark.read.format("jdbc")
        .option("url", JDBC_URL)
        .option("driver", JDBC_DRIVER)
        .option("user", JDBC_USER)
        .option("password", JDBC_PASSWORD)
        .option("dbtable", f"({query}) AS src")
        .load()
    )

    total = df.count()
    print(f"[{table_name}] Registros incrementais lidos do Postgres: {total}")

    if total > 0 and "change_op" in df.columns:
        dist = df.groupBy("change_op").count().collect()
        for row in dist:
            print(f"[{table_name}] change_op={row['change_op']} -> {row['count']} registros")

    return df


def write_landing_csv(df: DataFrame, table_name: str, batch_id: str) -> str:
    landing_path = f"{LANDING_CSV_BASE_PATH}/{table_name}/batch_id={batch_id}"

    df_to_write = (
        df
        .withColumn("landing_batch_id", F.lit(batch_id))
        .withColumn("landing_load_ts", F.lit(now_utc()))
    )

    total_to_write = df_to_write.count()
    print(f"[{table_name}] Registros a serem gravados em CSV: {total_to_write}")

    if total_to_write > 0 and "change_op" in df_to_write.columns:
        dist = df_to_write.groupBy("change_op").count().collect()
        for row in dist:
            print(f"[{table_name}] (CSV) change_op={row['change_op']} -> {row['count']} registros")

    (
        df_to_write.write
        .mode("append")
        .option("header", "true")
        .option("delimiter", ";")
        .csv(landing_path)
    )

    print(f"[{table_name}] Landing CSV gravada em: {landing_path}")
    return landing_path


def run_landing_for_table(table_name: str) -> None:
    if table_name not in TABLE_CONFIGS:
        raise ValueError(f"Tabela '{table_name}' não está configurada em TABLE_CONFIGS.")

    conf = TABLE_CONFIGS[table_name]

    print(f"================ INÍCIO LANDING: {table_name} ================")

    last_wm = get_watermark(table_name)
    print(f"[{table_name}] Watermark atual: {last_wm}")

    df_src = extract_incremental_from_postgres(table_name, conf, last_wm)

    if is_df_empty(df_src):
        print(f"[{table_name}] Nenhuma linha nova/atualizada desde o último watermark.")
        update_watermark(table_name, last_wm)
        print(f"================ FIM LANDING (sem dados): {table_name} ================")
        return

    batch_id = datetime.now().strftime("%Y%m%d%H%M%S")

    write_landing_csv(df_src, table_name, batch_id)

    new_max_data_ref = df_src.agg(F.max("data_ref").alias("max_dr")).collect()[0]["max_dr"]
    update_watermark(table_name, new_max_data_ref)

    print(f"[{table_name}] Novo watermark: {new_max_data_ref}")
    print(f"================ FIM LANDING: {table_name} ================")


init_schema(META_SCHEMA)
init_watermark_table()

for tbl in TABLE_CONFIGS.keys():
    run_landing_for_table(tbl)
